In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [4]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, scale!, evaluate, ncoeff
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using Roots
using NLsolve

┌ Info: Precompiling AdaptiveTransportMap [bdf749b0-1400-4207-80d3-e689c0e3f03d]
└ @ Base loading.jl:1260


### Hessian computation 

In [92]:
Nx = 2

m = 10

Ne = 8
ens = EnsembleState(Nx, Ne)

ens.S .=  [0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]';

X = ens.S

B = MultiBasis(CstProHermite(m; scaled =true), Nx)

idx = [0 0; 0 1; 1 0; 1 1; 2 1]
truncidx = idx[1:2:end,:]
Nψ = 5

coeff =   [0.6285037650645056;
 -0.4744029092496623;
  1.1405280011620331;
 -0.7217760771894809;
  0.11855056306742319]

f = ExpandedFunction(B, idx, coeff)

fp = ParametricFunction(f)
R = IntegratedFunction(fp)
H = HermiteMapk(R; α = 0.0)
S = Storage(H.I.f, X; hess = true);

In [93]:
hess_coeff(R, X)

8×5×5 Array{Float64,3}:
[:, :, 1] =
  0.0  -0.0   0.0  -0.0  -0.0
  0.0  -0.0   0.0  -0.0  -0.0
  0.0  -0.0   0.0  -0.0   0.0
  0.0  -0.0   0.0  -0.0   0.0
  0.0  -0.0   0.0  -0.0   0.0
  0.0  -0.0   0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0

[:, :, 2] =
 -0.0   0.00941367   -0.0   0.00584056    0.00156137
 -0.0   0.00115406   -0.0   0.00070505    0.000257328
 -0.0   8.75635e-5   -0.0   3.77278e-5   -4.66666e-5
 -0.0   0.00345292   -0.0   0.000760682  -0.00156138
 -0.0   0.000393109  -0.0   0.000246237  -4.49297e-5
 -0.0   0.00446468   -0.0   0.0027023     0.0011158
 -0.0  -0.00347744   -0.0  -0.00133303   -0.001884
 -0.0  -0.0043377    -0.0  -0.00236538   -0.00181341

[:, :, 3] =
  0.0  -0.0   0.0  -0.0  -0.0
  0.0  -0.0   0.0  -0.0  -0.0
  0.0  -0.0   0.0  -0.0   0.0
  0.0  -0.0   0.0  -0.0   0.0
  0.0  -0.0   0.0  -0.0   0.0
  0.0  -0.0   0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0
 -0.0  -0.0  -0.0  -0.0  -0.0

[:, :, 4] =
 -0.0   0.00584056   -0

In [94]:
J = 0.0
dJ = zeros(Nψ)
d2J = zeros(Nψ, Nψ)
J, dJ, d2J = hess_negative_log_likelihood!(J, dJ, d2J, coeff, S, H, X)

issymmetric(reshape2_cacheintegral[l, :, :]) = true
issymmetric(reshape2_cacheintegral[l, :, :]) = true
issymmetric(reshape2_cacheintegral[l, :, :]) = true
issymmetric(reshape2_cacheintegral[l, :, :]) = true
issymmetric(reshape2_cacheintegral[l, :, :]) = true
issymmetric(reshape2_cacheintegral[l, :, :]) = true
issymmetric(reshape2_cacheintegral[l, :, :]) = true
issymmetric(reshape2_cacheintegral[l, :, :]) = true


(1.8118825457682413, [1.064003062151197, 0.6384320024103914, 0.5751383902922512, 0.34446523801085754, -0.042992554511516186], [1.0 0.5760063173037648 … 0.29041239725778395 0.030077755319995653; 0.5760063173037648 0.3415780776279095 … 0.17220638450093798 0.015056997608376074; … ; 0.29041239725778395 0.17220638450093798 … 0.09323417550159312 0.014924813038570933; 0.030077755319995653 0.015056997608376074 … 0.014924813038570933 0.04821099540854864])

In [68]:
@time negative_log_likelihood!(J, dJ, coeff, S, H, X)

  0.000169 seconds (793 allocations: 47.328 KiB)


2.137129544913297

### Greedy fit

In [147]:
Nx = 10

m = 20

Ne = 1000
# ens = EnsembleState(Nx, Ne)

# ens.S .=  [0.267333   1.43021;
#           0.364979   0.607224;
#          -1.23693    0.249277;
#          -2.0526     0.915629;
#          -0.182465   0.415874;
#           0.412907   1.01672;
#           1.41332   -0.918205;
#           0.766647  -1.00445]';
X = ens.S
X = randn(Nx, Ne).^2 + 0.1*randn(Nx, Ne)
# idx = [0 0; 0 1; 1 0; 0 2; 2 0; 1 1; 2 1; 1 2; 1 3; 2 2]

# Nψ = 10

# coeff = [     1.0174962905416995;
#              -1.1871356902490515;
#              -1.9837596947482932;
#               0.5088563113730136;
#              -0.9948770215816068;
#               1.1697089852843865;
#              -0.4364897348115606;
#              -0.45195264571030846;
#               0.6717865772395687;
#               0.8540132401727857]

In [148]:
L = LinearTransform(X; diag = true)
transform!(L, X)

10×1000 Array{Float64,2}:
 -0.690202   -0.595004    1.53707   …  -0.591998    -0.578563   1.52853
 -0.441275   -0.314905   -0.282054     -0.30521     -0.622784   0.690694
  1.15504    -0.652401   -0.618832     -0.0466972   -0.4555    -0.691359
  3.07661    -0.686147   -0.726459     -0.764043     0.390474   0.0371078
 -0.323856   -0.237024    0.167072     -0.611194    -0.624197   3.35201
  1.53622    -0.640471   -0.284045  …  -0.0834316   -0.310722  -0.291826
 -0.131834    0.0363916   0.112176     -0.601238    -0.554947  -0.451961
 -0.352505    1.71733    -0.753808      0.469905    -0.345014  -0.141933
 -0.0696934   0.0338294  -0.336468     -0.00743629   1.11219   -0.688029
 -0.452564   -0.711301   -0.498921     -0.512323     0.983083  -0.73837

In [149]:
X_train = X[:,1:800]
X_valid = X[:,801:1000]

10×200 Array{Float64,2}:
 -0.733241  -0.588017   -0.570597   …  -0.591998    -0.578563   1.52853
 -0.504441   2.32238    -0.669967      -0.30521     -0.622784   0.690694
  0.63604   -0.596369   -0.220347      -0.0466972   -0.4555    -0.691359
 -0.428104  -0.767242   -0.0572687     -0.764043     0.390474   0.0371078
  0.287644  -0.37097     0.0700383     -0.611194    -0.624197   3.35201
 -0.102864   0.0141409  -0.687785   …  -0.0834316   -0.310722  -0.291826
 -0.232743  -0.752499   -0.475897      -0.601238    -0.554947  -0.451961
 -0.513879  -0.585413    0.42345        0.469905    -0.345014  -0.141933
 -0.400458  -0.638978   -0.599365      -0.00743629   1.11219   -0.688029
 -0.52674   -0.414987    2.33183       -0.512323     0.983083  -0.73837

In [152]:
@code_warntype greedyfit(m, Nx, X_train, X_valid, 15; verbose = false);

Variables
  #unused#::Core.Compiler.Const(AdaptiveTransportMap.var"#greedyfit##kw"(), false)
  @_2::NamedTuple{(:verbose,),Tuple{Bool}}
  @_3::Core.Compiler.Const(AdaptiveTransportMap.greedyfit, false)
  m::Int64
  k::Int64
  X::Array{Float64,2}
  Xvalid::Array{Float64,2}
  maxterms::Int64
  maxpatience::Int64
  verbose::Bool
  @_11::Int64
  @_12::Bool

Body::Tuple{HermiteMapk{_A,_B,_C} where _C where _B where _A,Array{Float64,1},Array{Float64,1}}
1 ─ %1  = Base.haskey(@_2, :maxpatience)::Core.Compiler.Const(false, false)
└──       goto #3 if not %1
2 ─       Core.Compiler.Const(:(Base.getindex(@_2, :maxpatience)), false)
│         Core.Compiler.Const(:(%3 isa AdaptiveTransportMap.Int64), false)
│         Core.Compiler.Const(:(%4), false)
│         Core.Compiler.Const(:(goto %9), false)
│         Core.Compiler.Const(:(%new(Core.TypeError, Symbol("keyword argument"), :maxpatience, AdaptiveTransportMap.Int64, %3)), false)
│         Core.Compiler.Const(:(Core.throw(%7)), false)
│         

In [151]:
@code_warntype greedyfit($m, $Nx, $X_train, $X_valid, $15; verbose = false);

ErrorException: syntax: "$" expression outside quote

In [89]:
@btime greedyfit($m, $Nx, $X_train, $X_valid, $10; verbose = false);

  1.001 s (1279836 allocations: 2.03 GiB)


In [99]:
Hk_old = HermiteMapk(m, Nx; α = 1e-6);

@time S = Storage(Hk_old.I.f, X);

Hk_new, train_error, valid_error = greedyfit(m, Nx, X_train, X_valid, 10; verbose = true);

  0.000038 seconds (42 allocations: 120.516 KiB)
1 terms - Training error: 1.4443504670380543, Validation error: 1.3147907978711455
2 terms - Training error: 1.013555754394048, Validation error: 1.0148284980423214
3 terms - Training error: 0.9740035731615574, Validation error: 0.9536709077333043
4 terms - Training error: 0.968194963388136, Validation error: 0.9269258934472301
5 terms - Training error: 0.8660654654774056, Validation error: 0.8538651976765612
6 terms - Training error: 0.8468938234481649, Validation error: 0.8374329264723278
7 terms - Training error: 0.8118015219715872, Validation error: 0.8007373200002506
8 terms - Training error: 0.808988833791454, Validation error: 0.8016700608672841
9 terms - Training error: 0.8055509920369043, Validation error: 0.8064967607768131
10 terms - Training error: 0.7857644650326624, Validation error: 0.8090534466320567


In [106]:
Hk_old = HermiteMapk(m, Nx; α = 1e-6);

@time S = Storage(Hk_old.I.f, X);

Hk_new, train_error, valid_error = greedyfit(m, Nx, X_train, X_valid, 10; verbose = true);

  0.000057 seconds (42 allocations: 120.516 KiB)
1 terms - Training error: 1.41428818328749, Validation error: 1.435039932873401
2 terms - Training error: 1.011647231546337, Validation error: 1.0765340323252046
3 terms - Training error: 0.954187562314565, Validation error: 1.009496573370243
4 terms - Training error: 0.9532824496069529, Validation error: 1.007776307368724
5 terms - Training error: 0.841380513349966, Validation error: 0.9041126563257862
6 terms - Training error: 0.8299467025145106, Validation error: 0.9037778557535633
7 terms - Training error: 0.8018842783083129, Validation error: 0.8750517929963674
8 terms - Training error: 0.801758584142784, Validation error: 0.8767880843281514
9 terms - Training error: 0.8015539210804201, Validation error: 0.8774101595696208
10 terms - Training error: 0.7995583928098936, Validation error: 0.8823916723449652


In [83]:
Hk_test = deepcopy(Hk_new)
setcoeff!(Hk_test, zero(getcoeff(Hk_new)));

In [85]:
S_test = Storage(Hk_test.I.f, X_train)
coeff_test = getcoeff(Hk_test)

res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S_test, Hk_test, X_train)), coeff_test, Optim.BFGS())
Optim.minimizer(res)

10-element Array{Float64,1}:
 -11.217886066730607
  11.50914577979369
  26.61111083798624
 -24.71937861667548
  34.997147393519235
 -17.03501613771925
  11.077321531464895
   0.034843907277600714
   0.0381866111200113
   0.1782552001411759

In [78]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, Hk_old, ens.S)), coeff, Optim.BFGS())
coeffopt = Optim.minimizer(res)

1-element Array{Float64,1}:
 -1.144117388080984